In [1]:
from datetime import datetime, timedelta, date
from dateutil.relativedelta import relativedelta
import pandas as pd
from pandasql import sqldf
import streamlit as st
import pull_nrg_data
import json
import http.client
import certifi
import ssl
import os
import timeit
import pickle
import altair as alt
import pytz
import numpy as np
tz = pytz.timezone('America/Edmonton')

In [11]:
def release_token(accessToken):
    path = '/api/ReleaseToken'
    server = 'api.nrgstream.com'
    headers = {'Authorization': f'Bearer {accessToken}'}
    context = ssl.create_default_context(cafile=certifi.where())
    conn = http.client.HTTPSConnection(server,context=context)
    conn.request('DELETE', path, None, headers)
    res = conn.getresponse()

with open('./default_pickle.pickle', 'rb') as handle:
    default_pickle = pickle.load(handle)

release_token(default_pickle['accessToken'])

In [12]:
t = datetime.now(pytz.timezone('America/Edmonton')).date() - timedelta(days=8)
t2 = datetime.now(pytz.timezone('America/Edmonton')) - timedelta(days=8)
default_pickle = {
    'accessToken':'',
    'current_data':'',
    'daily_outage_dfs':[0]*7,
    'monthly_outage_dfs':[0]*7,
    'alert_dates':{
        'Biomass & Other':t, 
        'Coal':t,
        'Dual Fuel':t,
        'Energy Storage':t,
        'Hydro':t,
        'Natural Gas':t,
        'Solar':t,
        'Wind':t,
        'Intertie':t
    }
}
with open('./default_pickle.pickle', 'wb') as handle:
    pickle.dump(default_pickle, handle, protocol=pickle.HIGHEST_PROTOCOL)
# Add default current values to dict
with open('./current_df.pickle', 'rb') as handle:
    current_df = pickle.load(handle)
default_pickle['current_data'] = current_df
# Add default monthly values to dict
with open('./monthly_df.pickle', 'rb') as handle:
    monthly = pickle.load(handle)
for i in range(7):
    default_pickle['monthly_outage_dfs'][i] = (t2,monthly)
# Add default daily values to dict
with open('./daily_df.pickle', 'rb') as handle:
    daily = pickle.load(handle)
for i in range(7):
    default_pickle['daily_outage_dfs'][i] = (t2,daily)
with open('./default_pickle.pickle', 'wb') as handle:
    pickle.dump(default_pickle, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [61]:
with open('./monthly_df.pickle', 'rb') as handle:
    monthly = pickle.load(handle)
monthly[3]

(datetime.datetime(2022, 9, 30, 9, 31, 44, 435659, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>),
                    timeStamp  value         fuelType
 9  2022-10-01 00:00:00-06:00    470             Coal
 10 2022-11-01 00:00:00-06:00     50             Coal
 11 2022-12-01 00:00:00-07:00      0             Coal
 9  2022-10-01 00:00:00-06:00   1270      Natural Gas
 10 2022-11-01 00:00:00-06:00    740      Natural Gas
 ..                       ...    ...              ...
 4  2024-05-01 00:00:00-06:00     30  Biomass & Other
 5  2024-06-01 00:00:00-06:00     30  Biomass & Other
 6  2024-07-01 00:00:00-06:00     30  Biomass & Other
 7  2024-08-01 00:00:00-06:00     30  Biomass & Other
 8  2024-09-01 00:00:00-06:00     30  Biomass & Other
 
 [192 rows x 3 columns])

In [49]:
len(monthly)

5

In [20]:
with open('./alert_dates.pickle', 'wb') as handle:
    pickle.dump(default_pickle['alert_dates'], handle, protocol=pickle.HIGHEST_PROTOCOL)

In [50]:
with open('./monthly_df.pickle', 'wb') as handle:
    pickle.dump(monthly, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [12]:
with open('./daily_df.pickle', 'rb') as monthly:
    monthly_df = pickle.load(monthly)
monthly_df[5]

(datetime.datetime(2022, 9, 26, 20, 43, 38, 233194, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>),
                    timeStamp  value              fuelType
 0  2022-09-26 00:00:00-06:00    0.0              Intertie
 1  2022-09-26 01:00:00-06:00    0.0              Intertie
 2  2022-09-26 02:00:00-06:00    0.0              Intertie
 3  2022-09-26 03:00:00-06:00    0.0              Intertie
 4  2022-09-26 04:00:00-06:00    0.0              Intertie
 ..                       ...    ...                   ...
 72 2022-09-29 00:00:00-06:00    0.0  3-Day Solar Forecast
 73 2022-09-29 01:00:00-06:00    0.0  3-Day Solar Forecast
 74 2022-09-29 02:00:00-06:00    0.0  3-Day Solar Forecast
 75 2022-09-29 03:00:00-06:00    0.0  3-Day Solar Forecast
 76 2022-09-29 04:00:00-06:00    0.0  3-Day Solar Forecast
 
 [26829 rows x 3 columns])